<a href="https://colab.research.google.com/github/shazzad-hasan/practice-deep-learning-with-pytorch/blob/main/gan/dcgan_celebrity_faces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# upload kaggle API key from your local machine
from google.colab import files
files.upload()

In [ ]:
# make a kaggle dir, copy the API key to it
# and make sure the file is only readable by yourself (chmod 600)
!mkdir ~/.kaggle 
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# use API command to download the dataset
!kaggle datasets download -d jessicali9530/celeba-dataset

In [ ]:
!unzip -qq celeba-dataset.zip

In [ ]:
# import required libraries
import torch
import torchvision

import os
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# check if cuda is available
train_on_gpu = torch.cuda.is_available()

if train_on_gpu:
  print("CUDA is available!")
else:
  print("CUDA is not available!")

device = torch.device("cuda") if train_on_gpu else torch.device("cpu")

In [ ]:
# settings
    
def set_all_seeds(seed):
    os.environ["PL_GLOBAL_SEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

random_seed = 125
set_all_seeds(random_seed)

### Load and visualiza dataset

In [ ]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

# load and transform data using ImageFolder
transform = transforms.Compose([
                  transforms.Resize(64),
                  transforms.CenterCrop(64),
                  transforms.ToTensor(),
                  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_data = datasets.ImageFolder(root="./celeba", transform=transform)

num_workers = 0
batch_size = 20

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)

In [ ]:
# visualize a batch of training data

def imshow(img):
  # unnormalize
  img = img / 2 + 0.5
  # convert from Tensor to image
  plt.imshow(np.transpose(img, (1,2,0)))

# obtain one batch on training images
dataiter = iter(train_loader)
images, labels = dataiter.next()
# convert images to numpy for display
images = images.numpy() 

# display 10 images
fig = plt.figure(figsize=(10,4))
for ind in np.arange(10):
  ax = fig.add_subplot(2, 10/2, ind+1, xticks=[], yticks=[])
  imshow(images[ind])

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# Discriminator network

class Discriminator(nn.Module):
  def __init__(self, ngpu):
      super(Discriminator, self).__init__()
      self.ngpu = ngpu
      self.main = nn.Sequential(
          # input size in_channels x 64 x 64
          nn.Conv2d(in_channels, num_feat_maps_d, 4, 2, 1, bias=False),
          nn.LeakyReLU(0.2, inplace=True),
          # state size. (num_feat_maps_d) x 32 x 32
          nn.Conv2d(num_feat_maps_d, num_feat_maps_d * 2, 4, 2, 1, bias=False),
          nn.BatchNorm2d(num_feat_maps_d * 2),
          nn.LeakyReLU(0.2, inplace=True),
          # state size. (num_feat_maps_d*2) x 16 x 16
          nn.Conv2d(num_feat_maps_d * 2, num_feat_maps_d * 4, 4, 2, 1, bias=False),
          nn.BatchNorm2d(num_feat_maps_d * 4),
          nn.LeakyReLU(0.2, inplace=True),
          # state size. (num_feat_maps_d*4) x 8 x 8
          nn.Conv2d(num_feat_maps_d * 4, num_feat_maps_d * 8, 4, 2, 1, bias=False),
          nn.BatchNorm2d(num_feat_maps_d * 8),
          nn.LeakyReLU(0.2, inplace=True),
          # state size. (num_feat_maps_d*8) x 4 x 4
          nn.Conv2d(num_feat_maps_d * 8, 1, 4, 1, 0, bias=False),
          nn.Sigmoid()
      )

  def forward(self, input):
      return self.main(input)

In [ ]:
# Generator network

class Generator(nn.Module):
  def __init__(self, ngpu):
      super(Generator, self).__init__()
      self.ngpu = ngpu
      self.main = nn.Sequential(
          # latent vertor z, going into a convolution
          nn.ConvTranspose2d(latent_dim, num_feat_maps_g * 8, 4, 1, 0, bias=False),
          nn.BatchNorm2d(num_feat_maps_g * 8),
          nn.ReLU(True),
          # state size. (num_feat_maps_g*8) x 4 x 4
          nn.ConvTranspose2d(num_feat_maps_g * 8, num_feat_maps_g * 4, 4, 2, 1, bias=False),
          nn.BatchNorm2d(num_feat_maps_g * 4),
          nn.ReLU(True),
          # state size. (num_feat_maps_g*4) x 8 x 8
          nn.ConvTranspose2d(num_feat_maps_g * 4, num_feat_maps_g * 2, 4, 2, 1, bias=False),
          nn.BatchNorm2d(num_feat_maps_g * 2),
          nn.ReLU(True),
          # state size. (num_feat_maps_g*2) x 16 x 16
          nn.ConvTranspose2d(num_feat_maps_g * 2, num_feat_maps_g, 4, 2, 1, bias=False),
          nn.BatchNorm2d(num_feat_maps_g),
          nn.ReLU(True),
          # state size. (num_feat_maps_g) x 32 x 32
          nn.ConvTranspose2d(num_feat_maps_g, in_channels, 4, 2, 1, bias=False),
          nn.Tanh()
          # state size. (in_channels) x 64 x 64
      )

  def forward(self, input):
      return self.main(input)

In [ ]:
## Discriminator hyparameters

# num of channels in the training images
in_channels = 3
# size of feature map in discriminator
num_feat_maps_d = 64


## Generator hyperparameters

# size of the latent vector, z
latent_dim = 100
# size of feature map in generator
num_feat_maps_g = 64

# Number of GPUs available (0 for CPU mode)
num_gpu = 1

# Build complete network
D = Discriminator(num_gpu).to(device)
G = Generator(num_gpu).to(device)

print(D)
print()
print(G)

In [ ]:
import torch.optim as optim

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# BCELoss function
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, latent_dim, 1, 1, device=device)

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

# optimizer for discriminator
d_optimizer = optim.Adam(D.parameters(), lr, betas=(beta1, 0.999))
# optimizer for generator
g_optimizer = optim.Adam(G.parameters(), lr, betas=(beta1, 0.999))

In [ ]:
import torchvision.utils as vutils

# Training Loop

# Number of training epochs
num_epochs = 5

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(train_loader, 0):

        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        D.zero_grad()
        # Format batch
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        # Forward pass real batch through D
        output = D(real_cpu).view(-1)
        # Calculate loss on all-real batch
        errD_real = criterion(output, label)
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        # Generate fake image batch with G
        fake = G(noise)
        label.fill_(fake_label)
        # Classify all fake batch with D
        output = D(fake.detach()).view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, label)
        # Calculate the gradients for this batch, accumulated (summed) with previous gradients
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Compute error of D as sum over the fake and the real batches
        errD = errD_real + errD_fake
        # Update D
        d_optimizer.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        G.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = D(fake).view(-1)
        # Calculate G's loss based on this output
        errG = criterion(output, label)
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        g_optimizer.step()

        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(train_loader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(train_loader)-1)):
            with torch.no_grad():
                fake = G(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1